In this notebook we'll try to figure out if we can use nibabel to create and save a cifiti image, which can be visuzlized using the connectome workbench.
For example if you use the Glasser or Cole Anticivec parcellation, they are in cifti format. It would be good if you can dump values onto each parcel/ROI and visualize them.


In [ ]:
import nibabel as nib # we use version
print("nibabel version:" + nib.__version__)
import numpy as np
import pandas as pd

Let us start by loading an example CIFTI file. We will use cifti files from the Cole Anticivec parcellation repository: 
https://github.com/ColeLab/ColeAnticevicNetPartition \
In this repo there are also workbench "scene" files you can use to visualize stuff in the connectome workbench

We will play around with two different types of cifti files \
dscalar, which dumps a scalar value to each surface vertex and each volume voxle (hence "dense") \
dlabel, which dumps a label integer value to each vertex/voxel, this is the "parcel ROI file" \

These files will be loaded as cifti2 object with nibabel

In [ ]:
template_cifti_dlabelfile = nib.load("../ColeAnticevicNetPartition/CortexSubcortex_ColeAnticevic_NetPartition_wSubcorGSR_parcels_LR.dlabel.nii")
template_cifti_dscalarfile = nib.load("../ColeAnticevicNetPartition/CortexSubcortex_ColeAnticevic_NetPartition_wSubcorGSR_parcels_LR_ReorderedByNetworks.dscalar.nii")


we can load the data assoicated with each object and exaime its content

In [ ]:
template_cifti_dscalarfile.get_fdata().shape

In [ ]:
len(np.unique(template_cifti_dlabelfile.get_fdata()))

So there are 718 unique ROIs in the dlabel file from the Cole Anticvic parcellation. How do we know what each integer represents? They provided a text file to label those.

In [ ]:
label_df = pd.read_csv("../ColeAnticevicNetPartition/CortexSubcortex_ColeAnticevic_NetPartition_wSubcorGSR_parcels_LR_LabelKey.txt", sep = "\t")

In [ ]:
label_df.head()

In [ ]:
#Here are all the thalamus parcels
label_df.loc[label_df['LABEL'].str.contains('Thalamus')]
print(len(label_df.loc[label_df['LABEL'].str.contains('Thalamus')]))

now let us write a fake value, say -1000, to all the thalamus voxels to a new dscalar file, using the thalamus parcel labels from the dlabel cifti file to index their locations. 

In [ ]:
#need to reload to link to new data object
template_cifti_dlabelfile = nib.load("../ColeAnticevicNetPartition/CortexSubcortex_ColeAnticevic_NetPartition_wSubcorGSR_parcels_LR.dlabel.nii")
new_label_data = template_cifti_dlabelfile.get_fdata()

In [ ]:
for key in label_df.loc[label_df['LABEL'].str.contains('Thalamus')]['KEYVALUE']:
    print(key)
    new_label_data[0,new_label_data[0,:]==key]=-1000

now we will create a new cifti dscalr file with this new_label_data, by using the header info from a cifti dscalar file as a template, and replace the data object with new_label_data \
https://nipy.org/nibabel/reference/nibabel.cifti2.html#nibabel.cifti2.cifti2.Cifti2Image


In [ ]:
new_cifti = nib.cifti2.cifti2.Cifti2Image(new_label_data, template_cifti_dscalarfile.header)
nib.save(new_cifti, 'new_cifti.dscalar.nii')

In workbench the thalamus is in dark blue

![](test1.png)

Now let us replace each thalamus parcel with a unique value. We will do this in the order of the parcels listed in Cole/Anticivec's ROI list. Imagin that you have a vector of data that are correctly ordered (say from the thalamocortical FC matrix), you can use this logic to display custom values in the correct anatomical location. The key is you have to make sure the order of your custom thalamic data matches the parcel order in the Cole Anticivec ROI parcel file.

In [ ]:
template_cifti_dlabelfile = nib.load("../ColeAnticevicNetPartition/CortexSubcortex_ColeAnticevic_NetPartition_wSubcorGSR_parcels_LR.dlabel.nii")
label_data = template_cifti_dlabelfile.get_fdata()
new_label_data = label_data.copy() # create copy so not to link to original copy
new_label_data[new_label_data!=0] = 0

i = 1
for key in label_df.loc[label_df['LABEL'].str.contains('Thalamus')]['KEYVALUE']:
    new_label_data[0,label_data[0,:]==key]=i
    i = i+1
    

In [ ]:
np.unique(new_label_data)

In [ ]:
#save and visualize
new_cifti = nib.cifti2.cifti2.Cifti2Image(new_label_data, template_cifti_dscalarfile.header)
nib.save(new_cifti, 'new_cifti.dscalar.nii')

In workbench each thalamus parcel has a unique color. And notice that everything else got set to 0.

![](test2.png)

So in the Cole/Anticivec parcellation there are 38 unique parcels in the thalamus. Here we relabeled them using the original ascending order in the dlabel cifti file, and visualize those integer values. As long as you have a vector of 38 thalamic parcel values that you are sure matches the  order in the original dlabel parcellation file, you can then write custom cifti files using the logic described above to visualize those values.